In [ ]:
# 定义使用哪一个回调函数，结果保存在数组
callbacks = [......]

# 我们在使用回调的时候，之中只需要在训练函数中指定回调即可：
model.fit(..., ..., callbacks=callbacks)

In [ ]:
'''
    2. 模型保存回调
    模型保存的回调函数为
    
    path: 保存模型的路径；
    monitor: 用哪个指标来评价模型的好坏，默认是验证集上的损失；
    verbose: 输出日志的等级，只能为 0 或 1；
    save_best_only: 是否只保存最好的模型，模型的好坏由 monitor 指定；
    save_weights_only: 是否只保存权重，默认 False ，也就是保存整个模型；
    save_freq: 保存的频率，可以为 ‘Epoch’ 或者一个整数，
                默认为每个 Epoch 保存一次模型；
                若是一个整数N，则是每训练 N 个 Batch 保存一次模型。
'''
tf.keras.callbacks.ModelCheckpoint(
    path, monitor='val_loss', verbose=0, save_best_only=False,
    save_weights_only=False, save_freq='epoch')

'''
3. 学习率回调

其中 verbose 参数仍然是日志输出的等级，默认为 0 ；
而 schedule 则是一个函数，用来定义一个学习率的变化。
'''
tf.keras.callbacks.LearningRateScheduler(
    schedule, verbose=0
)

# 其中 schedule 函数的一个示例如下所示：
# 学习率回调是在 20 个 Epoch 之前学习率保持不变
# 而在 20 个 Epoch 之后，每个 Epoch 学习率变为原来的 0.1 。
def my_schedule(epoch, lr):
  if epoch < 20:
    return lr
  else:
    return lr * 0.1

'''
4. 自定义回调
    我们在使用回调的过程之中难免会遇到要自定义回调的情况，
    这时我们便需要编写类来继承 tf.keras.callbacks.CallBack 类，从而实现我们的自定义回调。


'''
# 使用其中两个简单的函数来做一个简单的示例
# 我们便可以在每次训练开始，以及每个 Epoch 开始之时进行输出日志
class MyCallback(tf.keras.callbacks.Callback):

    def on_epoch_begin(self, epoch, logs=None):
        print("Start epoch {}.".format(epoch))

    def on_train_begin(self, logs=None):
        print("Starting training.")


In [ ]:
# 在这里，我们将同时使用模型保存回调、学习率回调以及自定义回调来做一个简单的示例：
import tensorflow as tf

model = tf.keras.Sequential([
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(64, activation='relu'),
      tf.keras.layers.Dense(10)
])

lr = 0.01

# 这里在编译的时候指定了优化器的学习率是多少
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
    loss="mse"
)

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

# 控制学习率
def my_schedule(epoch, lr):
  print('Learning rate: ' + str(lr))
  if epoch < 5:
    return lr
  else:
    return lr * 0.1

lr_callback = tf.keras.callbacks.LearningRateScheduler(my_schedule)

save_model_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='/model/', save_weights_only=True, verbose=1,
    monitor='val_loss', mode='min', save_best_only=True)

class MyCallback(tf.keras.callbacks.Callback):

    def on_epoch_begin(self, epoch, logs=None):
        print("Start epoch {}.".format(epoch))

    def on_train_begin(self, logs=None):
        print("Starting training.")

model.fit(x_train, y_train,
    batch_size=64, epochs=10,
    validation_data=(x_test, y_test),
    callbacks=[MyCallback(), lr_callback, save_model_callback],
)

model.evaluate(x_test, y_test)